In [11]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
import time
import numpy as np
import pandas as pd

from sklearn.utils import shuffle

from keras.backend import backend
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, SimpleRNN, Dropout
#from keras.layers.embeddings import Embedding

print ("============================================================")
print (" RNN for demographic features          Muratova Anna        ")
print ("============================================================")


start_time = time.time()

df = pd.read_excel("table.xlsx")

df.dropna(inplace = True)   # remove empty (NaN) lines
df = shuffle(df, random_state=7)
print ("Input data shape =", df.shape)
size = df.shape[0]
t = int(size * 0.8)         # train part size

x = df.as_matrix()[:size, 0:4]
y = df.as_matrix()[:size, 4] 
del df

X = np.empty((x.shape[0], x.shape[1]), dtype='int32')

for d in range(x.shape[1]):        
    xs = list(set(x[:, d]))
    if d == 0:
        print ("Number of unique sequences =", len(xs), "\n")
    xd = {xs[i]: i for i in range(len(xs))}
    for l in range(x.shape[0]):
        X[l][d] = xd[x[l][d]]

#X = StandardScaler().fit_transform(X)

XF_train = X[0:t, 1:]           # features only
XF_test  = X[t:size, 1:]
#print XF_train[0:10]

yn = np.empty((size), dtype='int32')

ys = list(set(y[:]))
yd = {ys[i]: i for i in range(len(ys))}
for l in range(len(yn)):
        yn[l] = yd[y[l]]


y_train = yn[0:t]
y_test  = yn[t:size]

# all unique characters to the set
events = set()
for seq in x[:, 0]:
    for event in [seq]:
        events.add(event)

events = list(events)
#print events

event_to_id = {t:i+1 for i,t in enumerate(events)}
#print event_to_id

max_seq_len = 8
seq_events_numbered = np.zeros((x.shape[0], max_seq_len), dtype='int32')

for i in range(seq_events_numbered.shape[0]):
    for k in range(len([x[i][0]])):
        seq_events_numbered[i][k] = event_to_id[x[i][0][k]]

S_train = seq_events_numbered[0:t, :]       # train sequences
S_test  = seq_events_numbered[t:size, :]    # test  sequences


print ("Train features  data shape =", XF_train.shape, y_train.shape)
print( "Test  features  data shape =", XF_test.shape,  y_test.shape)
print ("\nData preprocessing time, sec =  %0.2f" % (time.time() - start_time))

print ("============================================================")

print ("Keras backend =", backend())
 
time_01 = time.time()
print ("\nRNN classification by features" )

features = Sequential()
features.add(Dense(100, activation="sigmoid", input_dim=5))
features.add(Dropout(0.1))
features.add(Dense(100, activation='sigmoid'))
features.add(Dense(1, activation="sigmoid"))
features.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
features.fit(XF_train, y_train, epochs=400, batch_size=100)

time_02 = time.time()
score, acc = features.evaluate(XF_test, y_test, verbose=0)

time_03 = time.time() 


 RNN for demographic features          Muratova Anna        
Input data shape = (310, 5)
Number of unique sequences = 21 



/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


TypeError: 'int' object is not subscriptable

In [3]:
df.tail()

,for Sequence_mining (full),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
1673,14,20,31,28,30
473,17,21,26,26,28
213,19,19,19,22,22
1586,17,19,19,20,21
1382,17,18,23,19,18


In [1]:
# this is the code from https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 156s 3ms/step - loss: 0.2716 - acc: 0.9160 - val_loss: 0.0709 - val_acc: 0.9778
Epoch 2/12
60000/60000 [==============================] - 148s 2ms/step - loss: 0.0929 - acc: 0.9719 - val_loss: 0.0449 - val_acc: 0.9836
Epoch 3/12
60000/60000 [==============================] - 142s 2ms/step - loss: 0.0692 - acc: 0.9798 - val_loss: 0.0386 - val_acc: 0.9860
Epoch 4/12
60000/60000 [==============================] - 143s 2ms/step - loss: 0.0578 - acc: 0.9827 - val_loss: 0.0346 - val_acc: 0.9878
Epoch 5/12
60000/60000 [==============================] - 152s 3ms/step - loss: 0.0494 - a

In [2]:
# ...include code from https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

import shap
import numpy as np

# select a set of background examples to take an expectation over
background = x_train[np.random.choice(x_train.shape[0], 100, replace=False)]

# explain predictions of the model on three images
e = shap.DeepExplainer(model, background)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(x_test[1:5])


In [3]:
shap_values

[array([[[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         ...,
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]]],
 
 
        [[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         ...,
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]]],
 
 
        [[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         ...,
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]]],
 
 
        [[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         ...,
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]]]]), array([[[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         ...,
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]]],
 
 
        [[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         ...,
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],


In [4]:
A = [6,12,42]

In [21]:
print(A)

from itertools import combinations


def power_set(List):
    PS = [list(j) for i in range(len(List)) for j in combinations(List, i+1)]
    return PS

#print(power_set(A))
PS = power_set(A)
for i in PS:
    print(i)
    
def shapley_values_calculation(N):
    shapley_values = []
    n = 3
    for i in range(n):
        shapley = 0
        for j in N:
            if i not in j:
                cmod = len(j)
                Cui = j[:]
                bisect.insort_left(Cui,i)
                l = N.index(j)
                k = N.index(Cui)
                temp = float(float(characteristic_function[k]) - float(characteristic_function[l])) *\
                               float(math.factorial(cmod) * math.factorial(n - cmod - 1)) / float(math.factorial(n))
                shapley += temp
                    # if i is 0:
                    #     print j, Cui, cmod, n-cmod-1, characteristic_function[k], characteristic_function[l], math.factorial(cmod), math.factorial(n - cmod - 1), math.factorial(n)

        cmod = 0
        Cui = [i]
        k = N.index(Cui)
        temp = float(characteristic_function[k]) * float(math.factorial(cmod) * math.factorial(n - cmod - 1)) / float(math.factorial(n))
        shapley += temp

        shapley_values.append(shapley)

    return shapley_values
print(shapley_values_calculation(A))

[6, 12, 42]
[6]
[12]
[42]
[6, 12]
[6, 42]
[12, 42]
[6, 12, 42]


TypeError: argument of type 'int' is not iterable

In [20]:
#!/usr/bin/env python
from itertools import combinations
import math
import bisect
import sys

def power_set(List):
    PS = [list(j) for i in range(len(List)) for j in combinations(List, i+1)]
    return PS

def main():
    if len(sys.argv) < 2:
        print ('Usage: ./shapley.py <filename>')
        return

    characteristic_function = []

    with open(sys.argv[1]) as gamefile:
        for line in gamefile:
            try:
                n = int(line[line.index('=') + 1:])
            except:
                print("Invalid file format")
                sys.exit(0)
            break
        for line in gamefile:
            characteristic_function = line.strip().split(",")

    if n == 0:
        print("No players, exiting")
        sys.exit(0)

    tempList = list([i for i in range(n)])
    N = power_set(tempList)

    shapley_values = []
    for i in range(n):
        shapley = 0
        for j in N:
            if i not in j:
                cmod = len(j)
                Cui = j[:]
                bisect.insort_left(Cui,i)
                l = N.index(j)
                k = N.index(Cui)
                temp = float(float(characteristic_function[k]) - float(characteristic_function[l])) *\
                           float(math.factorial(cmod) * math.factorial(n - cmod - 1)) / float(math.factorial(n))
                shapley += temp
                # if i is 0:
                #     print j, Cui, cmod, n-cmod-1, characteristic_function[k], characteristic_function[l], math.factorial(cmod), math.factorial(n - cmod - 1), math.factorial(n)

        cmod = 0
        Cui = [i]
        k = N.index(Cui)
        temp = float(characteristic_function[k]) * float(math.factorial(cmod) * math.factorial(n - cmod - 1)) / float(math.factorial(n))
        shapley += temp

        shapley_values.append(shapley)

    print(shapley_values)

if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: '-f'

In [29]:
N = [[0], [1], [2], [3], [0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3], [0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3], [0, 1, 2, 3]]
n =4 
characteristic_function=['1', ' 1', ' 2', ' 1', ' 3', ' 4', ' 4', ' 3', ' 4', ' 4', ' 5', ' 7', ' 6', ' 6', ' 8']
print(N,n)

[[0], [1], [2], [3], [0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3], [0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3], [0, 1, 2, 3]] 4


In [36]:
shapley_values = []
for i in range(n):
    shapley = 0
    for j in N:
        if i not in j:
            cmod = len(j)
            Cui = j[:]
            bisect.insort_left(Cui,i)
            l = N.index(j)
            k = N.index(Cui)
            print(Cui)
            temp = float(float(characteristic_function[k]) - float(characteristic_function[l])) *\
                           float(math.factorial(cmod) * math.factorial(n - cmod - 1)) / float(math.factorial(n))

    cmod = 0
    Cui = [i]
    k = N.index(Cui)
    temp = float(characteristic_function[k]) * float(math.factorial(cmod) * math.factorial(n - cmod - 1)) / float(math.factorial(n))
    shapley += temp

    shapley_values.append(shapley)

print (shapley_values)

[0, 1]
[0, 2]
[0, 3]
[0, 1, 2]
[0, 1, 3]
[0, 2, 3]
[0, 1, 2, 3]
[0, 1]
[1, 2]
[1, 3]
[0, 1, 2]
[0, 1, 3]
[1, 2, 3]
[0, 1, 2, 3]
[0, 2]
[1, 2]
[2, 3]
[0, 1, 2]
[0, 2, 3]
[1, 2, 3]
[0, 1, 2, 3]
[0, 3]
[1, 3]
[2, 3]
[0, 1, 3]
[0, 2, 3]
[1, 2, 3]
[0, 1, 2, 3]
[0.25, 0.25, 0.5, 0.25]


In [35]:
print(characteristic_function,N)

['1', ' 1', ' 2', ' 1', ' 3', ' 4', ' 4', ' 3', ' 4', ' 4', ' 5', ' 7', ' 6', ' 6', ' 8'] [[0], [1], [2], [3], [0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3], [0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3], [0, 1, 2, 3]]


In [32]:
characteristic_function

['1',
 ' 1',
 ' 2',
 ' 1',
 ' 3',
 ' 4',
 ' 4',
 ' 3',
 ' 4',
 ' 4',
 ' 5',
 ' 7',
 ' 6',
 ' 6',
 ' 8']

In [33]:
shapley_values

[0.25, 0.25, 0.5, 0.25]

In [39]:
print(x_train, y_train, x_test, y_test)

[[[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]


 [[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]


 [[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.

In [43]:
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [48]:
print(x_train.shape)

(60000, 28, 28, 1)


In [5]:


# Create your first MLP in Keras
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

print(X,"\n",Y)
# create model
# model = Sequential()
# model.add(Dense(12, input_dim=8, activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# # Compile model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# # Fit the model
# model.fit(X, Y, epochs=150, batch_size=10)
# # evaluate the model
# scores = model.evaluate(X, Y)
# print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]
 [  1.     93.     70.    ...  30.4     0.315  23.   ]] 
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0.
 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1.
 1. 0

In [1]:
from sklearn.neural_network import MLPClassifier
MLPClassifier?

In [3]:
import panda as pd
fillna?

ModuleNotFoundError: No module named 'request'